# Extensión de MIDI con MelodyRNN de Magenta

Este notebook extiende un archivo MIDI ya existente mediante el modelo preentrenado MelodyRNN de Magenta (models/pretrained_models/basic_rnn.mag). A partir de este notebook construimos el script melodyrnn.py. Usamos este script para probar a llamarlo desde un script LUA de ReaScipt pero no conseguimos que funcionara. El script LUA de prueba se encuentra en proyecto_reaper/Scripts/prueba-nofunciona.lua

In [1]:
from note_seq import midi_file_to_note_sequence
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from note_seq.protobuf import generator_pb2
import note_seq

/home/carmenfdez/MUS/melodIA/venv/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/carmenfdez/MUS/melodIA/venv/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/carmenfdez/MUS/melodIA/venv/lib/python3.7/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn(

In [2]:
# Ruta al archivo MIDI exportado desde Reaper
input_midi_path = "../../Media/original.mid"

# Convertimos el archivo MIDI a NoteSequence (formato usado por Magenta)
try:
    input_sequence = midi_file_to_note_sequence(input_midi_path)
    print("MIDI cargado correctamente. Número de notas:", len(input_sequence.notes))
except Exception as e:
    print("Error al cargar el archivo MIDI:", e)

MIDI cargado correctamente. Número de notas: 25


In [4]:
# Rutas
bundle_path = "../../pretrained_models/basic_rnn.mag"
output_path = "../../Media/original_extended.mid"

# Cargar modelo
bundle = sequence_generator_bundle.read_bundle_file(bundle_path)
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map['basic_rnn'](checkpoint=None, bundle=bundle)
melody_rnn.initialize()

num_steps = 128 # change this for shorter or longer sequences
temperature = 1.0 # the higher the temperature the more random the sequence.

# Set the start time to begin on the next step after the last note ends.
last_end_time = (max(n.end_time for n in input_sequence.notes)
                  if input_sequence.notes else 0)
qpm = input_sequence.tempos[0].qpm 
seconds_per_step = 60.0 / qpm / melody_rnn.steps_per_quarter
total_seconds = num_steps * seconds_per_step

generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = temperature
generate_section = generator_options.generate_sections.add(
  start_time=last_end_time + seconds_per_step,
  end_time=total_seconds)

generated_sequence = melody_rnn.generate(input_sequence, generator_options)

note_seq.sequence_proto_to_midi_file(generated_sequence, output_path)

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from /tmp/tmpidwsfbcx/model.ckpt


2025-05-16 19:54:03.124294: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-16 19:54:03.124483: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-05-16 19:54:03.124550: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2025-05-16 19:54:03.124584: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2025-05-16 19:54:03.124617: W tensorf

INFO:tensorflow:Beam search yields sequence with log-likelihood: -157.333054 
